In [1]:
from common import *
from pandas.api.types import CategoricalDtype
from numba import njit

from sklearn.preprocessing import QuantileTransformer as QT
from scipy.stats import norm
ppf = norm.ppf

In [7]:
from given import *


# ------------------------------> looking at `W`

### Scratcher

In [8]:
#pd.to_pickle((F,P), '/big/data/saves/train_5searchbase.32.pkl')
F,P = pd.read_pickle('/big/data/saves/train_5fixedsince+1.32.pkl')

In [ ]:
F.index = P.index = F.index.map(lambda x: '{:x}'.format(x))

In [68]:
from sklearn.model_selection import GroupShuffleSplit

ho = P.quarter>=2015.5
ho.name = None
tr, cv = next(GroupShuffleSplit(n_splits=5, test_size=.5, random_state=44).split(F[~ho], P[~ho], groups=P[~ho].quarter))
_dummy = pd.Series(range(len(P)),index=P.index)
#tr, cv = _dummy.isin(tr), _dummy.isin(cv)

In [71]:
F.index[tr]

Index(['b2e8c', 'b2e8d', 'b2e8e', 'b2e8f', 'b2e90', 'b2e91', 'b2e92', 'b2e93',
       'b2e94', 'b2e95',
       ...
       '321a76', '321a77', '321a78', '321a79', '321a7a', '321a7b', '321a7c',
       '321a7d', '321a7e', '321a7f'],
      dtype='object', length=1346646)

In [72]:
guy = GroupShuffleSplit(n_splits=6, test_size=.5, random_state=44).split(F[~ho], P[~ho], groups=P[~ho].quarter)
for tr, cv in guy:
    plist(P.quarter[_dummy.isin(tr)].unique(), end=',\n')
    plist(P.quarter[_dummy.isin(cv)].unique(), end='\n\n')

[2009.0,2009.75,2010.75,2011.5,2012.25,2012.5,2012.75,2013.0,2013.25,2013.75,2014.0,2014.25,2015.0],
[2009.25,2009.5,2010.0,2010.25,2010.5,2011.0,2011.25,2011.75,2012.0,2013.5,2014.5,2014.75,2015.25]

[2009.25,2009.5,2011.0,2011.25,2011.5,2011.75,2012.0,2012.25,2013.0,2013.5,2014.5,2014.75,2015.0],
[2009.0,2009.75,2010.0,2010.25,2010.5,2010.75,2012.5,2012.75,2013.25,2013.75,2014.0,2014.25,2015.25]

[2011.0,2011.75,2012.0,2012.25,2012.75,2013.25,2013.5,2014.0,2014.25,2014.5,2014.75,2015.0,2015.25],
[2009.0,2009.25,2009.5,2009.75,2010.0,2010.25,2010.5,2010.75,2011.25,2011.5,2012.5,2013.0,2013.75]

[2009.5,2010.0,2010.25,2010.5,2011.0,2011.75,2012.25,2012.5,2013.0,2013.5,2014.25,2014.75,2015.25],
[2009.0,2009.25,2009.75,2010.75,2011.25,2011.5,2012.0,2012.75,2013.25,2013.75,2014.0,2014.5,2015.0]

[2009.75,2010.0,2010.25,2010.75,2011.75,2012.0,2012.25,2012.5,2013.0,2013.5,2013.75,2014.5,2015.0],
[2009.0,2009.25,2009.5,2010.5,2011.0,2011.25,2011.5,2012.75,2013.25,2014.0,2014.25,2014.75,2015.

### stock subsets

In [12]:
Mt = M[M.time>=train_start_time]

In [12]:
# # already in given.ipynb
# inUniCount = Mt.groupby('assetCodeId').universe.sum()
# stocksInUni = (inUniCount != 0).pipe(lambda x: x.index[x])
# assert stocksInUni.is_monotonic
# xStocksInUni = set(stocksInUni) #EDITCELL

In [18]:
W = O()
W.y = Mt[['time','assetCodeId','ooTEN']].pivot('time','assetCodeId').ooTEN
W.ay = Mt[['time','assetCodeId','aooTEN']].pivot('time','assetCodeId').aooTEN
W.u = Mt[['time','assetCodeId','universe']].pivot('time','assetCodeId').universe.fillna(0).astype(int)

In [22]:
C = O()
C.y = W.y.corr()
C.ay = W.ay.corr()
C.u = W.u.T @ W.u

In [23]:
@njit
def offdiag_items(ix, x, y):
    m, n = x.shape
    assert (m, n) == y.shape
    assert m == n
    assert (m,) == ix.shape
    t = (n**2-n) // 2
    I = np.full(t, -1, dtype=np.int64)
    J = np.full(t, -1, dtype=np.int64)
    X = np.full(t, 0, dtype=x.dtype)
    Y = np.full(t, 0, dtype=y.dtype)
    k = -1
    for i in range(m):
        for j in range(i+1, n):
            k += 1
            I[k] = ix[i]
            J[k] = ix[j]
            X[k] = x[i,j]
            Y[k] = y[i,j]
    return I, J, X, Y

In [24]:
C.v = pd.DataFrame(dict(zip([0, 1, 'Corr', 'Unic'], offdiag_items(C.y.index.values, C.y.values, C.u.values))))
C.av = pd.DataFrame(dict(zip([0, 1, 'Corr', 'Unic'], offdiag_items(C.ay.index.values, C.ay.values, C.u.values))))

In [ ]:
pd.to_pickle((O.py(W),O.py(C)), 'pair-working.pkl')

In [76]:
C.v = C.v[C.v.Unic!=0]
C.v.sort_values('Corr', ascending=False, inplace=True)
C.av = C.av[C.av.Unic!=0]
C.av.sort_values('Corr', ascending=False, inplace=True)

*see some stuff*

In [26]:
C.v[C.av.Unic>=1900].sort_values('Corr', ascending=False).Corr.pipe(lambda x: sum(x>.65))

1523

In [31]:
assetNameIdAssign.series.loc[3741], assetNameIdAssign.series.loc[3759]

('Syneos Health Inc', 'Varonis Systems Inc')

#### making pair feature table

In [ ]:
F['time'] = P.time
F_ = F[['time','assetCodeId','oo1','oo10','oo(15-5)','aoo1','aoo10','aoo(15-5)']]
F_['assetCodeId'] = F_.assetCodeId.astype(int)

In [ ]:
X = F_.pivot('time','assetCodeId')
X.columns = X.columns.swaplevel()
X = X.sort_index(axis=1)
X.iloc[:12,:12]

In [82]:
/list pd.DataFrame([[4,5,6,2,2],[1,4,9,8,1]],columns=[' a $',r' \\ !','\t','\n','_']).itertuples()

[Pandas(Index=0, _1=4, _2=5, _3=6, _4=2, _5=2),
 Pandas(Index=1, _1=1, _2=4, _3=9, _4=8, _5=1)]

In [78]:
def make_G(*, F, Cv, cutoff):
    X = F_.pivot('time','assetCodeId')
    X.columns = X.columns.swaplevel()
    X = X.sort_index(axis=1)
    for 

,0,1,Corr,Unic
2590169,1000,3388,1.0,12
1322534,477,2551,1.0,13
2169896,819,2521,1.0,6
582837,198,2396,1.0,13
1691457,620,2966,1.0,6


# ------------------------------> looking at `N`